In [1]:
import os
import torch
import torch.nn as nn
import pandas as pd
import lightning as L
import numpy as np

import g_main
import module_model

shots = 100000

config = g_main.json_config["config"]
config["batch_size"]   = 300
config["rnd_seed"]     = 0
config["num_pt_ptcs"]  = 4
config["num_bin_data"] = 25 # 25 * 10 * 2 * (1-0.8) = 100 test data
L.seed_everything(config["rnd_seed"])

ibmq_dir = "./ibmq"
os.makedirs(ibmq_dir, exist_ok=True)

#--------------------------------------------------------------------------
#                         FastJet release 3.4.0
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


Global seed set to 0


In [2]:
class IBMQQCGNN(module_model.QCGNN):
    def forward(self, x):
        x = self.net(x)
        x = torch.unflatten(x, dim=-1, sizes=(2**self.num_ir_qubits, self.num_nr_qubits))
        x = x.mT
        meas = x.detach()
        x = torch.sum(x, dim=-1) * self.scale
        return x, meas

class QuantumRotQCGNN(nn.Module):
    def __init__(self, num_ir_qubits, num_nr_qubits, num_layers, num_reupload, quantum_config):
        super().__init__()
        if "qiskit" in quantum_config["qdevice"]:
            ctrl_enc = lambda _input, control_values: g_main.qiskit_encoding(_input, control_values, num_ir_qubits, num_nr_qubits)
        else:
            ctrl_enc = lambda _input, control_values: g_main.pennylane_encoding(_input, control_values, num_ir_qubits, num_nr_qubits)
        self.phi = IBMQQCGNN(num_ir_qubits, num_nr_qubits, num_layers, num_reupload, ctrl_enc=ctrl_enc, shots=shots, **quantum_config)
        self.mlp = module_model.ClassicalMLP(in_channel=num_nr_qubits, out_channel=1, hidden_channel=0, num_layers=0)
    
    def forward(self, x):
        # inputs should be 1-dim for each data, otherwise it would be confused with batch shape
        x = torch.flatten(x, start_dim=-2, end_dim=-1)
        x, meas = self.phi(x)
        x = self.mlp(x)
        return x.detach(), meas

In [3]:
qidx = int(np.ceil(np.log2(config["num_pt_ptcs"])))
qnn, gl, gr  = 9, 1, 9
data_configs = [
    {"sig": "VzToZhToVevebb", "bkg": "VzToQCD", "abbrev":"BB-QCD", "cut": (800, 1000), "bin":10, "subjet_radius":0, "num_bin_data":config["num_bin_data"], "num_pt_ptcs":config["num_pt_ptcs"]},
    {"sig": "VzToTt", "bkg": "VzToQCD", "abbrev":"TT-QCD", "cut": (800, 1000), "bin":10, "subjet_radius":0, "num_bin_data":config["num_bin_data"], "num_pt_ptcs":config["num_pt_ptcs"]},
]

def load_state_dict(model, ckpt_path):
    old_state_dict = torch.load(ckpt_path)["state_dict"]
    new_state_dict = {}
    for old_key in old_state_dict.keys():
        new_key = old_key[6:]
        print(f"ModelLog: {old_key} ---> {new_key}")
        new_state_dict[new_key] = old_state_dict[old_key]
    model.load_state_dict(new_state_dict)

def prediction(x, quantum_config):
    # load model
    model_name   = QuantumRotQCGNN.__name__
    model_suffix = f"qidx3_qnn{qnn}_gl{gl}_gr{gr}"
    ckpt_key  = f"{model_name}_{model_suffix} | {data_config['abbrev']}_cut{data_config['cut']}"
    ckpt_path = g_main.get_ckpt(ckpt_key)
    model     = QuantumRotQCGNN(qidx, qnn, gl, gr, quantum_config)
    load_state_dict(model, ckpt_path)
    model.eval()
    return model(x)

result = {}
ibmq_backend = input("Enter IBMQ backend = ")
for data_config in data_configs:
    data_module = g_main.generate_datamodule(data_config, graph=False)
    assert len(data_module.test_dataloader()) == 1, "Check batch size, require 1 batch only"
    for x, y_true in data_module.test_dataloader():
        y_penl, meas_penl = prediction(x, {"qdevice": "default.qubit", "diff_method": "best", "qbackend": ""})
        y_ibmq, meas_ibmq = prediction(x, {"qdevice": "qiskit.ibmq", "diff_method": "parameter-shift", "qbackend": ibmq_backend})
        y_qasm, meas_qasm = prediction(x, {"qdevice": "qiskit.ibmq", "diff_method": "parameter-shift", "qbackend": "ibmq_qasm_simulator"})
    channel_result = {"x":x, "y_true":y_true, "y_penl":y_penl, "meas_penl":meas_penl, "y_qasm":y_qasm, "meas_qasm":meas_qasm, "y_ibmq":y_ibmq, "meas_ibmq":meas_ibmq}
    result[data_config["abbrev"]] = channel_result

npy_file = os.path.join(ibmq_dir, f"{ibmq_backend}_r{config['rnd_seed']}-p_{config['num_pt_ptcs']}_n{config['num_bin_data']}_b{config['batch_size']}-{qnn}_{gl}_{gr}.npy")
np.save(npy_file, result, allow_pickle=True)

# DataLog: Now loading hdf5 file VzToZhToVevebb|c800_1000_r0.hdf5
# DataLog: Successfully loading hdf5 file VzToZhToVevebb|c800_1000_r0.hdf5

# DataLog: Now loading hdf5 file VzToQCD|c800_1000_r0.hdf5
# DataLog: Successfully loading hdf5 file VzToQCD|c800_1000_r0.hdf5


# DataLog: Max number of particles = 4

# ModelLog: ckpt found at ./ckpt/QuantumRotQCGNN_qidx3_qnn3_gl1_gr3 | BB-QCD_cut(800, 1000)_ptc8_bin10-500_R0 | 20231105_slurm_0/checkpoints/epoch=29-step=2400.ckpt
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (2, 0, 3)
ModelLog: model.phi.net.0.weights ---> phi.net.0.weights
ModelLog: model.mlp.net.weight ---> mlp.net.weight
ModelLog: model.mlp.net.bias ---> mlp.net.bias
# ModelLog: ckpt found at ./ckpt/QuantumRotQCGNN_qidx3_qnn3_gl1_gr3 | BB-QCD_cut(800, 1000)_ptc8_bin10-500_R0 | 20231105_slurm_0/checkpoints/epoch=29-step=2400.ckpt
# ModelLog: Quantum device  = qiskit.ibmq | Qubits (IR, WK, NR) = (2, 1, 3)
ModelLog: model.phi.net.0.weights ---> phi.net.0.w